# Youtube Video Summarizer & Insight Extractor

  Input: YouTube video link

  Backend:

    Download audio from YouTube (yt_dlp).

    Transcribe with OpenAI Whisper API.

    Summarize + extract insights with OpenAI GPT model.

  Output: Transcript + Summary + Insights in the UI

In [1]:
# --- Install dependencies ---
!pip install yt-dlp gradio openai ffmpeg-python transformers accelerate librosa``

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.4 MB/s eta 0:00:00


In [2]:
# --- Imports ---
import os
import yt_dlp
import gradio as gr
import torch
import librosa
from google.colab import userdata
from openai import OpenAI
from transformers import pipeline

In [3]:
# --- Get API key from Colab Secrets ---
openai_key = userdata.get('OPENAI')
if not openai_key:
    raise ValueError("No API key found. Please add your key in Colab secrets with the name 'OPENAI'.")

In [4]:
# Initialize OpenAI client (for GPT summarization/insights)
client = OpenAI(api_key=openai_key)

In [5]:

# --- Step 1: Download audio from YouTube ---
def download_audio(youtube_url, output_file="audio.wav"):
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": "temp_audio.%(ext)s",
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "wav",
                "preferredquality": "192",
            }
        ],
        "quiet": True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    return "temp_audio.wav"

In [6]:
# --- Step 2: Load Whisper ASR pipeline with chunking ---
device = 0 if torch.cuda.is_available() else -1
asr_pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3",
    device=device,
    chunk_length_s=30  # ensures long audio is chunked
)

def transcribe_audio(file_path):
    result = asr_pipe(file_path, return_timestamps=False)
    return result["text"]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [7]:
# --- Step 3: Summarize & Extract Insights using GPT ---
def analyze_transcript(transcript):
    prompt = f"""
      You are an AI assistant. Given the transcript below, perform the following tasks:

      1. Extract all questions asked in the transcript.

      2. Rewrite each answer or suggested answer as if responding in a professional job interview setting:

        - Use the STAR method (Situation, Task, Action, Result) for behavioural or experiential questions.

        - For technical questions, respond as a subject matter expert, demonstrating clarity, depth, and confidence.

      Ensure the responses are polished, concise, and tailored to impress an interviewer.

    Transcript:
    {transcript}
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [8]:
# --- Step 4: Save results as text files ---
def save_text(content, filename):
    path = f"/content/{filename}"
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    return path

In [9]:
# --- Full pipeline with file outputs ---
def process_video(youtube_url):
    try:
        # Step 1: Download audio
        audio_file = download_audio(youtube_url)

        # Step 2: Transcribe with Whisper
        transcript = transcribe_audio(audio_file)

        # Step 3: Analyze with GPT
        analysis = analyze_transcript(transcript)

        # Step 4: Save to files
        transcript_file = save_text(transcript, "transcript.txt")
        analysis_file = save_text(analysis, "summary_insights.txt")

        # Return both text + downloadable file paths
        return transcript, analysis, transcript_file, analysis_file
    except Exception as e:
        return f"Error: {str(e)}", "", None, None


In [10]:
# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# 🎬 YouTube Video Summarizer & Insight Extractor")
    gr.Markdown("Paste a YouTube link → get transcript (via Whisper) + AI-powered summary & insights (via GPT).")

    with gr.Row():
        youtube_url = gr.Textbox(label="YouTube Video URL")
        run_button = gr.Button("Process Video")

    with gr.Row():
        transcript_output = gr.Textbox(label="Transcript", lines=15)
        analysis_output = gr.Textbox(label="Summary & Insights", lines=10)

    with gr.Row():
        dl_transcript = gr.File(label="Download Transcript", visible=False)
        dl_analysis = gr.File(label="Download Summary & Insights", visible=False)

    # Update text + show file download links after processing
    run_button.click(
        fn=process_video,
        inputs=youtube_url,
        outputs=[transcript_output, analysis_output, dl_transcript, dl_analysis]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5224cc5480eb5751ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
